## Building Chatbots with Conversation History Using Langchain

In [323]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [324]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A4D9E2A000>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A4D9E80950>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [325]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi my name is malek abuhammad and I am a Junior Ai Engineer")])

AIMessage(content="Hello Malek Abuhammad!\n\nIt's great to meet you. Being a Junior AI Engineer is an exciting field to be in.  \n\nWhat are you working on these days?  Is there anything specific you'd like to discuss or learn about related to AI?  \n\nI'm happy to chat about anything from machine learning algorithms to the ethical implications of AI.  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 24, 'total_tokens': 106, 'completion_time': 0.149090909, 'prompt_time': 0.004651758, 'queue_time': 0.213170094, 'total_time': 0.153742667}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7a5d42fd-0ae5-430e-a282-4a7484eb19bc-0', usage_metadata={'input_tokens': 24, 'output_tokens': 82, 'total_tokens': 106})

In [326]:
from langchain_core.messages import AIMessage

model.invoke(
  [
    HumanMessage(content="Hi my name is malek abuhammad and I am a Junior Ai Engineer"),
    AIMessage(content="Hello Malek Abuhammad, it's nice to meet you! \n\nThat's great that you're a Junior AI Engineer.  What are you currently working on or interested in within the field of AI? \n\nI'm always eager to learn more about what people are doing in this exciting field.\n"),
    HumanMessage(content="Hey what's my name and what do I do ?")
  ]
)

AIMessage(content="You are Malek Abuhammad, and you are a Junior AI Engineer!  \n\nI remember that from our first conversation. 😊  Is there anything else you'd like to tell me about yourself or your work?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 112, 'total_tokens': 159, 'completion_time': 0.085454545, 'prompt_time': 0.01967282, 'queue_time': 0.23057855400000002, 'total_time': 0.105127365}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c38f688e-91f6-497f-8b57-882a6f7458fc-0', usage_metadata={'input_tokens': 112, 'output_tokens': 47, 'total_tokens': 159})

## Message History

In [327]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
  if session_id not in store:
    store[session_id]=ChatMessageHistory()
  return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [328]:
config={"configurable":{"session_id":"chat1"}}

In [329]:
response=with_message_history.invoke(
  [HumanMessage(content="Hi my name is malek abuhammad and I am a Junior Ai Engineer")],
  config=config
)

response.content

"Hi Malek Abuhammad, it's nice to meet you! \n\nBeing a Junior AI Engineer is an exciting field to be in.  What are you most interested in working on right now?  \n\nDo you have any projects you're particularly proud of, or any challenges you're facing that you'd like to discuss? I'm happy to chat about anything AI-related!\n"

In [330]:
response=with_message_history.invoke(
  [HumanMessage(content="What's my name ?")],
  config=config
)

response.content

'Your name is Malek Abuhammad. \n\nI remember that from our introduction! 😊  Is there anything else I can help you with? \n'

In [331]:
config1={"configurable":{"session_id":"chat2"}}

In [332]:
response=with_message_history.invoke(
  [HumanMessage(content="What's my name ?")],
  config=config1
)

response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it!\n"

In [333]:
response=with_message_history.invoke(
  [HumanMessage(content="Hey my name is Ahmad")],
  config=config1
)

response.content

"Hi Ahmad, it's nice to meet you!  Is there anything I can help you with today?\n"

In [334]:
response=with_message_history.invoke(
  [HumanMessage(content="What's my name ?")],
  config=config1
)

response.content

'Your name is Ahmad. 😊  \n\nI remember that you told me earlier!  \n\nDo you have any other questions for me?\n'

## Prompt Templates

In [335]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
  [
    ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
    MessagesPlaceholder(variable_name="messages")
  ]
)

chain=prompt|model

In [336]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Malek")]})

AIMessage(content="Hello Malek, it's nice to meet you!\n\nI'm here to help. What can I do for you?  Ask me anything! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 32, 'total_tokens': 70, 'completion_time': 0.069090909, 'prompt_time': 0.002331354, 'queue_time': 0.159992473, 'total_time': 0.071422263}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--fedee0ef-a287-40fa-8acc-31af5a9390c8-0', usage_metadata={'input_tokens': 32, 'output_tokens': 38, 'total_tokens': 70})

In [337]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [338]:
config={"configurable":{"session_id":"chat3"}}

response=with_message_history.invoke(
  [HumanMessage(content="Hi my name is malek")],
  config=config
)

response.content

"Hi Malek, it's nice to meet you!  \n\nI'm happy to help with any questions you have. Just ask away! 😊  What can I do for you today? \n"

In [339]:
config={"configurable":{"session_id":"chat3"}}

response=with_message_history.invoke(
  [HumanMessage(content="whats my name")],
  config=config
)

response.content

'Your name is Malek!  I remember that from when you first introduced yourself.  😊  \n'

## Add more complexity

In [340]:
prompt=ChatPromptTemplate.from_messages(
  [
    ("system","You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
    MessagesPlaceholder(variable_name="messages")
  ]
)

chain=prompt|model

In [341]:
response=chain.invoke(
  {"messages":[HumanMessage(content="Hi my name is malek")],"language":"arabic"}
)

response.content

'مرحباً يا ملك،  😊 \n\nكيف يمكنني مساعدتك اليوم؟ \n'

In [342]:
with_message_history=RunnableWithMessageHistory(
  chain,
  get_session_history,
  input_messages_key="messages"
  )

In [343]:
config = {"configurable": {"session_id": "chat4"}}
response=with_message_history.invoke(
  {"messages":[HumanMessage(content="Hi I am malek")],"language":"arabic"},
  config=config
)

response.content

'مرحبا ملك، كيف حالك؟  😊  \n\n(Hello Malek, how are you?) \n'

In [344]:
response=with_message_history.invoke(
  {"messages":[HumanMessage(content="What's my name ?")],"language":"arabic"},
  config=config
)

response.content

'اسمك ملك. 😊\n\n(Your name is Malek.) \n'

## Managing the Conversation History


In [345]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
  max_tokens=50,
  strategy="last",
  token_counter=model,
  include_system=True,
  allow_partial=False,
  start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [346]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
  RunnablePassthrough.assign(messages=itemgetter("messages")|
                             trimmer)
  | prompt
  | model
)

response=chain.invoke(
  {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
  }
)

response.content

"As an AI, I don't have access to your personal information, including your taste preferences.  \n\nTo figure out your favorite ice cream, you could:\n\n* **Think back to your last few ice cream outings!** What did you choose?\n* **Consider your favorite flavors in other treats.** Do you like chocolate, fruity flavors, or something more unique?\n* **Browse an ice cream menu** and see what sounds most delicious! 🍦\n\nLet me know if you want to brainstorm some ice cream flavors! 😋\n"

In [347]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: "What is 2 + 2?"\n'

In [348]:
with_message_history=RunnableWithMessageHistory(
  chain,
  get_session_history,
  input_messages_key="messages"
  )

config={"configurable":{"session_id":"chat5"}}

In [349]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or personal information about you, so I don't know your name. \n\nWould you like to tell me? 😊  \n"

In [350]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I have no memory of past conversations.  \n\nIf you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n"

In [351]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
  max_tokens=100,
  strategy="last",
  token_counter=model,
  include_system=True,
  allow_partial=False,
  start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [352]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
  RunnablePassthrough.assign(messages=itemgetter("messages")|
                             trimmer)
  | prompt
  | model
)

response=chain.invoke(
  {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
  }
)

response.content

'You said you like vanilla ice cream! 😊  \n\n\n\n'

In [353]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked what 2 + 2 is. 😊 \n\n\n\n'

In [354]:
with_message_history=RunnableWithMessageHistory(
  chain,
  get_session_history,
  input_messages_key="messages"
  )

config={"configurable":{"session_id":"chat6"}}

In [355]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Bob! 😊 \n\n\n\n'

In [356]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked "whats 2 + 2".  😊  \n'